In [2]:
from collections import defaultdict, Counter
import re

with open('thresholdchunktweets.txt') as f:
    lines = f.read().splitlines()

kWindowTweets = defaultdict(Counter)
i = -1
new = True
for line in lines :
    if(line == 'CHUNKBREAK' or new):
        i += 1
        new = False
        kWindowTweets[(i)] = []
    else :
        kWindowTweets[(i)].append(line)
kWindowTweets

defaultdict(collections.Counter,
            {0: ['14:57:02 time:time Mumbai Indians without overseas players is enough to win ipl  INDvENG',
              '14:57:02 time:time INDvENG cricket 181-2 after 17 overs.',
              "14:57:03 time:time BCCI is inviting all England team to join under 14 bowling academy, it'll be very helpful for them..INDvENG",
              '14:57:05 time:time ENGvIND T20Cricket is at another level. This is why now @OfficialSLC do not deserve to play any form of cricket a ',
              '14:57:05 time:time @Cricketracker Adapting at every batting position. INDvENG',
              "14:57:07 time:time I want RohithSharma 's hands and eyes, handy to have some extra time.INDvENG",
              '14:57:11 time:time Kohli giving background music to every shot being played from the other end  INDvENG',
              '14:57:14 time:time I wish @hardikpandya7 stays still or goes leg side to get more room.. To pandya they are bowling only back of the l ',
       

In [3]:
import nltk
def is_proper_english(tweet,threshold = 3):
    """
       Checks whether the value of # of isalpha() words - # of proper english words is greater than a particular threshold
       Returns True if the tweet has considerable english words, else False
    """

    dict_obj = construct_dictionary() # word,count of each word in dictionary.txt
    words = nltk.word_tokenize(tweet) # list of words from the tweet
    scores = []
    
    for word in words:
        if word.isalpha():
            scores.append(dict_obj.get(word,0)) # score of 0 for anything not in the .txt
            
    if scores.count(0) > threshold:
        #print(tweet)
        return False
    else:
        return True
    
def construct_dictionary():
    dict_obj = {}
    f = open("dictionary.txt","r")
    lines = f.readlines()
    for line in lines:
        dict_obj[line.strip()] = dict_obj.get(line.strip(),1)

    return dict_obj

In [4]:
import nltk
import operator

stop_words = stopwords=['game','sss','sky','that','this','the','get','gets','ask','match','n\'t','and','good','well',
                                'engvsind','teamindia','let','like','need','got','hai','t20is','indian','indvseng',
                                'amp','best','most','one','ind','eng','india', 'england','t20','t20i','indveng','engvind',
                                'series','wow','woah','today','awesome','fuck','shit','yay','...','gone','come','going',
                               'need','koi','are','nervous','take','world','class','ground','can','ipl','please','someone',
                                'first','last','what','playing','back','king','great','see','indvseng_2021','kya','seal',
                               'ffs','congratulations','done','lord','man','cricket','now','getting','gon','5th','team',
                                'big','you','time','class','and','well','matches','seeing','find','much','getting','know',
                                'gone','another','again','nervous','back','.....','way','mumbai','no.1','say','gets','just'
                                ,'set','pant','hands']

def top_k_tweets(tweets, words_k, tweets_k):
        # get word scores (basically a frequency count)
        word_score = {} # dict that holds (word,score)
        for tweet in tweets:
            tweet=tweet.split("time:time")[1].lower()
            words = nltk.word_tokenize(tweet) # get all the words in the tweet
            for word in words:
                if len(word) > 2 and word not in stop_words:
                    word_score[word] = word_score.get(word,0)+1 # score of the word+=1, otherwise if new word: set to 1
                    
        sorted_word_score = sorted(word_score.items(), key=operator.itemgetter(1),reverse = True)
        
        #printing (word,score) in descending order
        #for i,val in enumerate(sorted_word_score):
        #print(val)
        
        count = 0
        top_k_words = []
        
        # Take the top_k words in to a separate list
        with open("topkwords_final2.txt","a") as f:
            for entry in sorted_word_score:
                if count > words_k: # iterate until you get 20 words
                    break
                top_k_words.append(entry[0])
                f.write("%s\n" % entry[0])
                count += 1
            f.write("\nCHUNKBREAK\n")
         
        #Calculating score of each tweet based on score of words in the tweet
        tweet_score = []
        for tweet in tweets:
            score = 0
            for word in set(nltk.word_tokenize(tweet)):
                if word in top_k_words:
                    score += sorted_word_score[top_k_words.index(word)][1]
            tweet_score.append((tweet,score))
            
        sorted_tweets = sorted(tweet_score, key=operator.itemgetter(1),reverse = True)
        
        count = 0
        with open('toptweets_final2.txt', 'w') as f:
            for (tweet,score) in sorted_tweets:
                if count > tweets_k:
                    break
                f.write("%s\n" % tweet)
                count += 1
            f.write("\n")
        
        result = [tweet for (tweet,score) in sorted_tweets]
        return result[:tweets_k]
    
def k_subsets_i(n, k):

    '''
    Yield each subset of size k from the set of intergers 0 .. n - 1
    n -- an integer > 0
    k -- an integer > 0
    '''
    # Validate args
    if n < 0:
        raise ValueError('n must be > 0, got n=%d' % n)
    if k < 0:
        raise ValueError('k must be > 0, got k=%d' % k)
    # check base cases
    if k == 0 or n < k:
        yield set()
    elif n == k:
        yield set(range(n))

    else:
        # Use recursive formula based on binomial coeffecients:
        # choose(n, k) = choose(n - 1, k - 1) + choose(n - 1, k)
        for s in k_subsets_i(n - 1, k - 1):
            s.add(n - 1)
            yield s
        for s in k_subsets_i(n - 1, k):
            yield s
            
def k_subsets(s, k):
    s = list(s)
    n = len(s)
    for k_set in k_subsets_i(n, k):
        yield set([s[i] for i in k_set])
        
def generate_nck(n,k):
    result = []
    for t in k_subsets(range(0,n),k):
        result.append(list(t))
    return result

def jaccard_similarity(tweet1, tweet2):
    """
        Returns the jaccard similarity between two tweets, disregarding all the stopwords
        N(A) n N(B)
        -----------
        N(A) u N(B)
    """
    words1 = []
    words2 = []
    for word in nltk.word_tokenize(tweet1):
        if len(word) > 2 and word not in stop_words:
            words1.append(word)

    for word in nltk.word_tokenize(tweet2):
        if len(word) > 2 and word not in stop_words:
            words2.append(word)


    words1 = set(words1)
    words2 = set(words2)
    
    return (len(list(words1&words2))+0.)/len(list(words1|words2))

def top_k_tweets_jaccard(tweets,k):
    """
        For each tweet, score is the sum of jaccard similarity of the tweet with the remaining tweets
        in the set.
        Now returns top k tweets which have the highest scores.
    """
    temp_scores = {}
    scores = {}

    for i in range(0,len(tweets)):
        for j in range(0,len(tweets)):
            if i!=j and not temp_scores.has_key(str(j)+":"+str(i)):
                temp_scores[str(i)+":"+str(j)] = jaccard_similarity(tweets[i],tweets[j])
                #print(i,j)
    
    #print "Scores calculated"

    for i in range(0,len(tweets)):
        score = 0
        for j in range(0,len(tweets)):
            if i<j:
                score += temp_scores.get(str(i)+":"+str(j),0)
            if i>j:
                score += temp_scores.get(str(j)+":"+str(i),0)
        scores[i] = score
                
    sorted_scores = sorted(scores.items(), key=operator.itemgetter(1),reverse = True)

    count = 0
    for (index,score) in sorted_scores:
        if count > k:
            break
        #print(tweets[index],score)
        count += 1

def refine_tweets_jaccard(tweets, no_of_tweets=2):
    """
        Generates nc2 pairs among the given n tweets and returns the tuple with 
        least jaccard_similarity among them.
    """
    
    pairs = generate_nck(len(tweets),no_of_tweets) # len(tweets) will be 20

    pair_score = []
    for pair in pairs: # find jaccard value between all the items in each combination
        score = 0
        for i in range(0,len(pair)):
            for j in range(i+1,len(pair)):
                p1 = tweets[pair[i]].split('time:time')[1]
                p2 = tweets[pair[j]].split('time:time')[1]
                score += jaccard_similarity(p1,p2)
                #print(i,j,jaccard_similarity(tweets[pair[i]],tweets[pair[j]]))
        #print(pair,score)
        pair_score.append((pair,score))

    sorted_pair_score = sorted(pair_score, key=operator.itemgetter(1))

    top_k_index = sorted_pair_score[0][0]
    result = []
    for i in range(0,len(top_k_index)):
        result.append(tweets[top_k_index[i]])
        #print(tweets[top_k_index[i]])
    return result

In [33]:
for chunknum in range(len(kWindowTweets)):
    topktweets=[]
    topktweets=(top_k_tweets(kWindowTweets[chunknum],20,20))
    tweets_per_moment = 2
    r = refine_tweets_jaccard(topktweets,tweets_per_moment)
    for tweet in r:
        print(re.sub('time:time','',tweet))

14:57:05  ENGvIND T20Cricket is at another level. This is why now @OfficialSLC do not deserve to play any form of cricket a 
14:57:07  I want RohithSharma 's hands and eyes, handy to have some extra time.INDvENG
15:01:53  INDvENGMark Wood hits 50 with ball as n batsmen smashed him all over the park ends his dismal spell at 53/0 in 4 overs!!
15:00:26  Making it difficult for pant. INDvENG
15:04:16  @ChloeAmandaB Wood 50 wood gets his half century off 24 balls. England clearly has plans this innings and wood is 
15:04:24  200 for Team.India INDvENG ENGvIND
15:07:58  Reaction from jofra to bowl 20th over against inform Hardik pandyaINDvENG 
15:08:20  English bowlers are just collecting 50's for fun...whilst bowling  INDvENG
15:10:25  A sales funnel is basically a system for generating consistent leads and sales from 
15:10:16  Virat Kohli opening and ending the innings INDvENGIndia
15:21:59  India registered highest team total against England Rohit-Virat opening partnership 94 runs boost 

In [8]:
topktweets=[]
topktweets=(top_k_tweets(kWindowTweets[0],20,20))
tweets_per_moment = 2
r2 = refine_tweets_jaccard(topktweets,tweets_per_moment)
for tweet in r2:
    print(re.sub('time:time','',tweet))

14:57:03  BCCI is inviting all England team to join under 14 bowling academy, it'll be very helpful for them..INDvENG
14:57:07  I want RohithSharma 's hands and eyes, handy to have some extra time.INDvENG


In [9]:
topktweets=[]
topktweets=(top_k_tweets(kWindowTweets[1],20,20))
tweets_per_moment = 2
r2 = refine_tweets_jaccard(topktweets,tweets_per_moment)
for tweet in r2:
    print(re.sub('time:time','',tweet))

15:01:53  INDvENGMark Wood hits 50 with ball as n batsmen smashed him all over the park ends his dismal spell at 53/0 in 4 overs!!
15:00:26  Making it difficult for pant. INDvENG


In [10]:
topktweets=[]
topktweets=(top_k_tweets(kWindowTweets[2],20,20))
tweets_per_moment = 2
r2 = refine_tweets_jaccard(topktweets,tweets_per_moment)
for tweet in r2:
    print(re.sub('time:time','',tweet))

15:04:16  @ChloeAmandaB Wood 50 wood gets his half century off 24 balls. England clearly has plans this innings and wood is 
15:04:24  200 for Team.India INDvENG ENGvIND
